# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [26]:
df = pd.read_csv(os.path.join("..","output","cities.csv"))
del df["Unnamed: 0"]
df

,City Name,Country Code,Date (UTC),Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
0,Tiksi,RU,2020-04-10 21:51:12,71.69,128.87,20.59,93,52,22.35
1,Celestun,MX,2020-04-10 21:52:48,20.87,-90.40,88.11,51,13,12.21
2,New Norfolk,AU,2020-04-10 21:51:27,-42.78,147.06,49.05,71,75,4.70
3,Luganville,VU,2020-04-10 21:56:03,-15.53,167.17,80.60,69,40,1.12
4,Vaini,TO,2020-04-10 21:50:04,-21.20,-175.20,78.80,69,40,8.05
...,...,...,...,...,...,...,...,...,...
543,Paracuru,BR,2020-04-10 21:56:39,-3.41,-39.03,84.04,74,20,8.05
544,Bakchar,RU,2020-04-10 21:58:24,57.02,82.07,29.05,94,30,6.33
545,Gondanglegi,ID,2020-04-10 21:58:24,-7.75,112.01,75.06,90,4,3.53
546,Balkhash,KZ,2020-04-10 21:56:55,46.84,74.98,37.72,63,0,11.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [97]:
# Store latitude and longitude in locations
locations = df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
rating = df["Humidity (%)"].astype(float)

In [103]:
gmaps.configure(api_key = g_key)

fig = gmaps.figure(zoom_level=2,center=(0,0))

# Plot Heatmap
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=40,
                                 point_radius=5)
# Add layer
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [104]:
narrowed_city_df = df.loc[(df["Temperature (F)"]>=68.00) & (df["Temperature (F)"]<=78.00) & (df["Cloudiness (%)"] <= 5) & (df["Wind Speed (MPH)"] <= 10.00)]
narrowed_city_df





,City Name,Country Code,Date (UTC),Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (MPH)
15,Tura,IN,2020-04-10 21:54:56,25.52,90.22,71.89,36,0,3.87
38,Curico,CL,2020-04-10 21:56:05,-34.98,-71.23,69.80,49,0,8.05
60,Bulandshahr,IN,2020-04-10 21:56:16,28.40,77.85,77.92,29,2,2.48
65,Dakar,SN,2020-04-10 21:54:55,14.69,-17.44,77.45,73,0,4.70
68,Reconquista,AR,2020-04-10 21:56:16,-29.15,-59.65,71.96,46,0,4.70
78,Port Hedland,AU,2020-04-10 21:53:13,-20.32,118.57,71.60,88,0,6.93
95,Santa Fe,AR,2020-04-10 21:54:04,-31.63,-60.70,72.86,45,0,1.99
178,Pereira Barreto,BR,2020-04-10 21:56:42,-20.64,-51.11,73.96,64,2,2.82
179,Sunland Park,US,2020-04-10 21:56:42,31.80,-106.58,75.76,12,1,4.70
214,Paka,MY,2020-04-10 21:56:54,4.64,103.44,73.20,100,2,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [105]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "keyword": "Hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in narrowed_city_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotels = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotels = hotels.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        narrowed_city_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

/opt/anaconda3/envs/PythonData2020_1/lib/python3.6/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/anaconda3/envs/PythonData2020_1/lib/python3.6/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [113]:
narrowed_city_df = narrowed_city_df.dropna(how='any')

In [114]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City Name}</dd>
<dt>Country Code</dt><dd>{Country Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Latitude", "Longitude"]]

In [115]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel Details: {name}" for name in hotel_info])
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))